In [ ]:
# TODO: 
# - write parameters into a text file to recall them later
# - create Constants class?
# - create gui:
#      * draw layout/contents/functionality on paper (do it like a pro instead of a blockhead for once)
#      * ???
#      * profit
# - brain vision recorder LSL
# - extend marker class?
# - stop experiment on keypress?
#

import random
import numpy as np

# not sure if this is useful
class Marker:
    
    ################ standard definitions ######################
    def __init__(self,identifier):
        self.value = identifier
        
    def __str__(self):
        return 'Marker ' + str(self.value)
    
    def __repr__(self):
        return 'Marker ' + str(self.value)
    
    ################# CONTENT            #######################
        
    def identify(self): return self.value

class Sequence:
    
        def __init__(self,probt,probsd,length,prenov):
            self.sequence = []
            self.expandedSeq = []
            self.markerCount = 0
            self.amount_Target = 0
            self.amount_Novelty = {}
            self.amount_Standard = 0
        
            self.trails(probt,probsd,length,prenov)
            if self.sequence == []:
                raise Exception
        
        def trails(self,Prob_T, Probs_D,Length,PreNov):

            amount_T = round(Length * Prob_T) # get amount of Targets from propability of a Target + amount of stimuli 
            amount_Ds = {} # fill dictonary with the amounts of every distractor stimuli same as above
            
            # this will be the sum of all probabilities but the standard one, so I can see if it 
            # is even possible to create a sequence
            probs_non_standard = Prob_T
            
            
            for name, prob in Probs_D.items():   
                amount = round(prob*Length)
                amount_Ds[name] = amount
                probs_non_standard += prob 

            if probs_non_standard * PreNov > 1: 
                print(self.errorMsg(probs_non_standard,PreNov))
                self.sequence = [] # TODO: raise exception here
                pass
            '''
            this happens if you for example say you want 2 standard stimuli previous to each novelty stimuli, the whole thing
            has to be 10 stimuli long and the two novelty stimuli have to appear with a probability of 0.4
            => length 10 => 8 novelty stimuli, each has to succeed 2 standard stimuli => 3 * 8 == 24 > 10!
            => that's simply not possible
            '''
            
            # save these values so we can call them outside of this function
            self.amount_Target = amount_T
            self.amount_Novelty = amount_Ds
            self.amount_Standard = Length - amount_T - sum(amount_Ds.values())

            sequence, amount_S = self.makeSeqs(PreNov,amount_T,amount_Ds,Length) 
            ''' 
            takes the amount of standard stimuli previous to novelty stimuli, the amount of target and distractor stimuli
            and the total amount of presented stimuli and returns a list of atomic sequences and the amount of standard stimuli
            which are still free to distribute
            '''

            self.sequence = self.permutations(amount_S * [['S']]  + sequence)
            '''
            we 'randomly' choose one permutation of the list of atomic sequences
            and return it
            '''


        def makeSeqs(self,pn,aT,aDs,length):

            seq_S = pn * ['S']

            seq_T = seq_S + ['T']
            seq_Ds = []

            for name, amount in aDs.items():
                seq_Ds += amount * [seq_S + [name]]

            result = seq_Ds + aT * [seq_T]
            return result, (length - (len(result) * (pn+1)))

        # returns one possible 'random' permutation of the given list
        def permutations(self,list):
            permus = random.sample(list,len(list))    
            return flatten(permus)

        # simply creates the error message when the given parameters in trails dont fit
        def errorMsg(self,probs,prenov):
            maxPreNov = round(1 / probs)
            maxProbs = float('%.2f'%(1 / prenov))
            return '\nIt is not possible to create a sequence of stimuli with the given probabilites for novelty stimuli and the given amount of buffer previous to novelty stimuli!\
                    \nEither change the buffer to {0} at max or the sum of novelty probabilites to {1} at max'.format(maxPreNov,maxProbs)
        
        
        # write oddball steps into a list, creating a sequence of actions which then can be 
        # easily executed
        # ISI = inter stimulus interval, PSI = pre stimulus interval, 
        # scaling_jitter = scaling factor for standard distribution, make it bigger!!!
        def expand(self,ISI,PSI,standard_dev,maxValue_jitter,minValue_jitter,mean_jitter,scaling_jitter):
            # I only call the expand_sequence method here, see below
            self.expandedSeq = self.expand_sequence(
                                               ISI,
                                               PSI,
                                               standard_dev,
                                               maxValue_jitter,
                                               minValue_jitter,
                                               mean_jitter,
                                               scaling_jitter)
            
        # maps te expand_single function over the whole sequence
        # since x consists of the pattern [[...],[...],...,[...]]
        # I have to flatten it into [.....]
        def expand_sequence(self,*args):
            x = map(lambda i: self.expand_single(i,*args),self.sequence)
            return flatten(list(x))

        # just concatinate the values 
        def expand_single(self,piece,ISI,PSI,sd,mav,miv,mean,scaling):
            realISI = ISI - PSI
            return ([self.marker()] +
                    [PSI]           + 
                    [self.marker()] + 
                    [piece]         +  
                    [self.marker()] + 
                    [realISI+self.calc_jitter(sd,mav,miv,mean,scaling)]
                   )

        # pulls a value from the gausian standard distribution, magnifies it with the 
        # scaling factor and then trunctuates it with the max and min values
        def calc_jitter(self,standard_dev, max_val, min_val, mean, scaling):
            return minmax(min_val,
                          max_val,
                          round(scaling*np.random.normal(mean,standard_dev)))
        
        # creates a Marker (see above, far above)
        def marker(self):
            curm = self.markerCount
            self.markerCount = curm+1
            return Marker(curm)
        
        ####### GETTERS ########
        
        def getTargetAmount(self):   return self.amount_Target
        def getStandardAmount(self): return self.amount_Standard
        def getNoveltyAmount(self):  return self.amount_Novelty        
        def getSeq(self):            return self.sequence
        def getExpandedSeq(self):    return self.expandedSeq
        def getMarkerCount(self):    return self.markerCount
        
# concatinates a list of lists
def flatten(list):
    return [e for sublist in list for e in sublist]
    # how to read it:
    # for each sublist in the given list, for each element in the sublist, put that element into a list

# rounds a value up or down if it is not in between the given min value and max value
# minmax(5,10,6) = 6; minmax(5,10,11) =  10
def minmax(minv,maxv,x):
    t1 = min(x,maxv)
    t2 = max(t1,minv)
    return t2
        
############################################################################################################
############################################################################################################

# Testing

# define parameters
target_probability = 0.1
distractor_probabilites = {
    'd1' : 0.1,
    'none': 0.1
} 
sequence_length = 20
prenovelty_standard_stimuli = 1

# create sequence 
new_sequence = Sequence(target_probability,
                        distractor_probabilites,
                        sequence_length,
                        prenovelty_standard_stimuli)
print(new_sequence.getSeq())

print('\n\n')

standard_dev = 1
mean_jitter = 0
max_jitter = 200
min_jitter = -200
scaling_jitter = 200
isi = 1500
psi = 500


new_sequence.expand(isi,psi,standard_dev,max_jitter,min_jitter,mean_jitter,scaling_jitter)

print(new_sequence.getExpandedSeq())




#######################################################################################################
#######################################################################################################


In [ ]:
# Here I implemented the sequence execution

from playsound import playsound # you need to install this library/ not a base library
import time
import os


# I inherit Sequence for structure
# inherit simply means that Execute has all the methods Sequence has plus some extra I define here
class Execute(Sequence):
    def __init__(self,soundDict,*args): # *args == probt,probsd,length,prenov. I was just too lazy to write it out
        super().__init__(*args) # here I call the constructor for Sequence with probt, probsd, lenght, prenov
        
        self.sound = soundDict # here I save all the sounds to be used
        
    # executes the sequence of actions
    def execute(self): 
        for action in self.getExpandedSeq():
            self.execute_action(action)    

    # execute a single action
    def execute_action(self,action):
        if isinstance(action,Marker): # in case it is a Marker object do this
            print('Sent marker %d to brain visualizer via LSL' % action.identify())
        elif isinstance(action,str): # String object aka a sound
            playsound(self.sound[action])
        elif isinstance(action,(int,float)): # int or float aka wait time
            time.sleep(action/1000) # time.sleep pauses the system for the number given in seconds, so I have to divide it by 1k
            
            
            
######################################################################################################### 
#######################   Testing
os.chdir('C:/Users/suuuj/Desktop/training room')
            
sounds = {
    'T' : 'baboon.mp3',
    'D1' : 'bark.mp3',
    'S' : 'batschatter.mp3'
}

probD = {
    'D1' : 0.1
}

standard_dev = 1
mean_jitter = 0
max_jitter = 200
min_jitter = -200
scaling_jitter = 200
isi = 1500
psi = 500

x = Execute(sounds,0.1,probD,20,1) # create Execute object

print(x.getSeq())

x.expand(isi,psi,standard_dev,max_jitter,min_jitter,mean_jitter,scaling_jitter) # create actions sequence

print(x.getExpandedSeq())

x.execute() # execute axtion sequence


In [26]:
from tkinter import * # you need to install this / not in base library
from tkinter import filedialog
import ntpath

def button(master,text,side=LEFT,command=None):
    w = Button(master,text=text,command=command)
    w.pack(expand=YES,side=side,fill=BOTH)
    return w

def labelentry(masterl,masterr,text):
    e = IntVar()
    Entry(masterr,borderwidth=5,textvariable=e).pack()
    Label(masterl,text=text,borderwidth=5).pack(anchor=W)
    return e
    

class Gui(Frame):
    
    def __init__(self):
        Frame.__init__(self)
        self.pack(expand=YES,fill=BOTH)
        self.master.minsize(1000,200)
        
        self.initParameters()
        self.initStimFrame()        
        self.initParamFrame()
        
    def initParameters(self):
        
        self.distCount = 0
        self.distDict = {}
        
        self.sounds = {}
        
        self.target_prob = 0
        
        self.length = 0
        
        self.curProb = 1
        self.isi = 0
        self.psi = 0
        self.pnss = 0
        self.jit_mean = 0
        self.jit_max = 0
        self.jit_min = 0
        self.jit_scaling = 1
        self.sd = 0
        
        self.noTarget = True
        self.noStandard = True
        
    def initStimFrame(self):
        
        stimulus_frame = Frame(self,relief=SUNKEN)
        stimulus_frame.pack(side=LEFT,expand=YES,fill=BOTH)        
        
        # Button Frame
        button_frame = Frame(stimulus_frame,relief=RAISED,borderwidth=5)
        button_frame.pack(fill=BOTH)  
        
        self.add_stim_but = button(button_frame,'Add Stimulus',command=self.addStim)
        self.remove_stim_but = button(button_frame,'Remove Stimulus')
        self.change_stim_but = button(button_frame,'Change Stimulus')
        
        
        # Listbox Frame
        lbf = Frame(stimulus_frame,borderwidth=5)
        lbf.pack(fill=BOTH)
        
        self.listbox = Listbox(lbf)
        self.listbox.pack(side=LEFT, expand=YES,fill=BOTH)        
        
        scrolly = Scrollbar(lbf,command=self.listbox.yview)
        scrolly.pack(side=RIGHT,fill=Y)
        
        
        
        self.listbox.configure(yscrollcommand = scrolly.set)
        
        # Length Entry
        
        length_frame = Frame(stimulus_frame,borderwidth=2,relief=RIDGE)
        length_frame.pack(fill=X,expand=YES)
        
        
        Label(length_frame,text='Length :').pack(side=LEFT,anchor=W)
        self.length_entry = IntVar()
        Entry(length_frame,textvariable=self.length_entry).pack(side=RIGHT,fill=X,expand=YES)
        
        # Info Entry
        
        self.ientry = StringVar()
        Entry(stimulus_frame,borderwidth=2,state=DISABLED,textvariable=self.ientry).pack(side=BOTTOM,fill=BOTH,expand=YES)
        
        
        
    def initParamFrame(self):
        parameter_frame = Frame(self,borderwidth=5,relief=SUNKEN)
        parameter_frame.pack(side=LEFT,expand=YES,fill=BOTH)
        
        # Label
        Label(parameter_frame,text='Parameters',relief=GROOVE,borderwidth=3).pack(expand=YES,fill=X)
        
        # Parameters        
        pf = Frame(parameter_frame)
        pf.pack(fill=BOTH,expand=YES)
        
        col_entry = Frame(pf)
        col_label = Frame(pf)
        
        col_label.pack(side=LEFT,fill=Y,expand=YES)
        col_entry.pack(side=RIGHT,fill=BOTH,expand=YES)
        
        self.isi_entry = labelentry(col_label,col_entry,'ISI')
        self.psi_entry = labelentry(col_label,col_entry,'PSI')
        self.pnss_entry = labelentry(col_label,col_entry,'Pre-Novelty S-Stimuli')
        
        self.jitter_sd_entry=labelentry(col_label,col_entry,'Jitter SD')
        self.jitter_scaling_entry=labelentry(col_label,col_entry,'Scaling Jitter')
        self.jitter_mean_entry = labelentry(col_label,col_entry,'Jitter Mean')
        self.jitter_max_entry = labelentry(col_label,col_entry,'Jitter Max')
        self.jitter_min_entry = labelentry(col_label,col_entry,'Jitter Min')
        
        # Connect + Start button
        
        cs = Frame(parameter_frame)
        cs.pack(side=BOTTOM,expand=YES,fill=X)
        
        self.start_but = button(cs,'Start Audio Sequence')
        self.start_but.config(state=DISABLED)
        self.connect_but = button(cs,'Initialize LSL outlet')
        
    
    
    def addStim(self):
        oWin = Toplevel(self)
    
        cr = Frame(oWin)
        cr.pack()
        
        # type listbox
        lbf = Frame(cr,relief=SUNKEN,borderwidth=5)
        lbf.pack(side=LEFT)
        lb = Listbox(lbf,height=3)
        
        lb.insert(END,'Distractor')
        if self.noTarget: lb.insert(END,'Target')
        if self.noStandard: lb.insert(END,'Standard')

        lb.pack()


        # probabilty entry
        ef = Frame(cr,borderwidth=5,relief=SUNKEN)
        ef.pack(side=LEFT,fill=BOTH)

        Label(ef,text='Probability',borderwidth=3).pack()
        prob = DoubleVar()
        Entry(ef,textvariable=prob).pack(side=BOTTOM)

        # open .mp3

        audio_frame = Frame(cr,relief=SUNKEN,borderwidth=5)
        audio_frame.pack(expand=YES,fill=BOTH)


        self.path = StringVar()
        
        Button(audio_frame,
               text='open sound file',
               borderwidth=3, 
               command = self.handleClickEvent
              ).pack(expand=YES, fill=BOTH)
        
        Entry(audio_frame,
              textvariable=self.path,
              state=DISABLED
             ).pack()

        # Create button

        Button(oWin,
               text='Create Stimulus',
               borderwidth=5,
               command = lambda : self.createStim(lb,prob,self.path,oWin)).pack(side=BOTTOM,expand=YES,fill=X)
        
    
    def handleClickEvent(self):
        self.path.set(filedialog.askopenfilename())
    
    def createStim(self,lb,prob,path,win):        
        k = 0
        try:
            stimType = lb.get(lb.curselection())
        except:
            self.error_win('Choose a stimuli type!')
            k = 1        
        
        try:
            probability = prob.get()
            if probability > self.curProb:
                self.error_win('Probability is to high. Maximum is %.1f' % self.curProb)
                k = 1
        except:
            self.error_win('Enter a probability!')
            k = 1
            
        try:
            filepath = self.path.get()
        except:
            self.error_win('Specify an audio file!')
            k = 1
        
        if k == 0:
            self.createEntry(stimType,probability,filepath)        

            self.listbox.insert(END,
                                'StimType: %s, Probability: %.1f, Audiofile: %s' 
                                % 
                                (stimType,probability,ntpath.basename(filepath))
                               )
            win.destroy()
            self.path.set('')
        
    def createEntry(self,stimType,prob,filepath):
        if stimType == 'Distractor':
            name = 'd%d' % self.distCount
            self.distCount += 1
            self.distDict[name] = prob
            self.sounds[name] = filepath
        elif stimType == 'Target':
            self.target_prob = prob
            self.sounds['T'] = filepath
            self.noTarget = False
        elif stimType == 'Standard':
            self.sounds['S'] = filepath
            self.noStandard = False
        self.curProb -= prob
        
    def error_win(self,msg):
        
        ewin = Toplevel(self)
        fr = Frame(ewin,borderwidth=5,relief=GROOVE)
        fr.pack(expand=YES,fill=BOTH)
        Label(fr,text=msg).pack()
        Button(fr,text='OK',command=ewin.destroy).pack()
        

        
        
        
        
        
Gui().mainloop()
    
